In [2]:
from transformers import BertTokenizer, BertForPreTraining
import torch

tokenizer = BertTokenizer.from_pretrained("./mbert_trained_mix_2.0/")
model = BertForPreTraining.from_pretrained("./mbert_trained_mix_2.0/")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

prediction_logits = outputs.prediction_logits
seq_relationship_logits = outputs.seq_relationship_logits

print(f'prediction_logits: {prediction_logits}')
print(f'seq_relationship_logits: {seq_relationship_logits}')

Some weights of the model checkpoint at ./mbert_trained_mix_2.0/ were not used when initializing BertForPreTraining: ['qa_outputs.bias', 'qa_outputs.weight']
- This IS expected if you are initializing BertForPreTraining from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForPreTraining from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForPreTraining were not initialized from the model checkpoint at ./mbert_trained_mix_2.0/ and are newly initialized: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predict

prediction_logits: tensor([[[-0.7960, -0.4499, -0.2590,  ..., -1.3657, -0.6401,  0.1440],
         [ 0.4767,  0.4180,  0.0518,  ...,  0.7086, -0.2624, -0.3882],
         [ 0.2944,  0.1217, -0.0426,  ..., -0.0133,  0.0677, -0.4058],
         ...,
         [ 0.3072,  0.4365, -0.0385,  ...,  0.3094, -0.3532, -0.4403],
         [ 0.0200, -0.2791, -0.5170,  ..., -0.4483, -0.6695, -1.1791],
         [-0.3772, -0.9417, -0.7060,  ..., -1.8003, -0.2143, -1.7891]]],
       grad_fn=<AddBackward0>)
seq_relationship_logits: tensor([[ 0.1569, -0.0778]], grad_fn=<AddmmBackward>)


In [ ]:
# https://qiita.com/m__k/items/e312ddcf9a3d0ea64d72

from transformers import (
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    AutoConfig
)


# https://huggingface.co/docs/transformers/main_classes/model#transformers.PreTrainedModel.from_pretrained
model = AutoModelForQuestionAnswering.from_pretrained(
    './mbert_trained_mix_2.0/'
)
tokenizer = AutoTokenizer.from_pretrained(
    './mbert_trained_mix_2.0/'
)
config = AutoConfig.from_pretrained(
    './mbert_trained_mix_2.0/'
)

# from transformers.trainer_utils import get_last_checkpoint
# last_checkpoint = get_last_checkpoint('./') # training_args.output_dir

# input the test_file
# Field object and TabularDataset object

from torchtext import data

TEXT = data.Field(tokenize=tokenizer)
LABEL = data.Field(sequential=False, use_vocab=False)

test_data = torchtext.data.TabularDataset.splits(
    path = './dataset/test_knowhow/',
    test = 'test_v2.4_job_neo.json',
    format = 'json',
    fields=[('Text', TEXT), ('label',LABEL)]
)

# BERTの順伝搬ときにoutput_attention = Trueで、Attention weightを取得できる
batch = next(iter(test_iter))

last_hidden_state, pooler_output, attentions = model(batch.Text[0], output_attentions=True)
print(last_hidden_state.size())
print(pooler_output_size())
print(len(attentions), attentions[-1].size())

# GPUの設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# prediction
from utils_qa import postprocess_qa_predictions

In [18]:
import run_qa

import torch
import logging

from transformers import (
    AutoModelForQuestionAnswering,
    AutoTokenizer,
    AutoConfig
)

# https://huggingface.co/docs/transformers/main_classes/model#transformers.PreTrainedModel.from_pretrained
model = AutoModelForQuestionAnswering.from_pretrained(
    './mbert_trained_mix_2.0/'
)
tokenizer = AutoTokenizer.from_pretrained(
    './mbert_trained_mix_2.0/'
)
config = AutoConfig.from_pretrained(
    './mbert_trained_mix_2.0/'
)

# GPUの設定
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

## 如何输入数据集？？？
from datasets import load_dataset, load_metric

raw_datasets = load_dataset(
    'json',
    data_files={'test':'./dataset/test_knowhow/test_v2.4_job_neo.json'},
    field='data',
    # data_args.dataset_config_name,
    # cache_dir=model_args.cache_dir,
    use_auth_token=True
)
column_names = raw_datasets['test'].column_names
question_column_name = "question"
context_column_name = "context"
answer_column_name = "answer"

#print(raw_datasets['test'][question_column_name])

# print(f'column_names:{column_names}')
# print(f'question_column_name:{question_column_name}')
# print(f'context_column_name:{context_column_name}')
# print(f'answer_column_name:{answer_column_name}')

# 从config中读取model的情报

question = raw_datasets['test'][question_column_name][1]
context = raw_datasets['test'][context_column_name][1]

print(question)
print(context)

inputs = tokenizer(
    question, 
    context,
    max_length=384,
    truncation=True,
    return_tensors='pt'
)
print(inputs.input_ids.size()) # [batch_size, sequence_length]
with torch.no_grad():
    outputs = model(**inputs, output_attentions=True)

#print(outputs)

answer_start_index = outputs.start_logits.argmax()
answer_end_index = outputs.end_logits.argmax()

predict_answer_tokens = inputs.input_ids[0, answer_start_index:answer_end_index+1]
answer_predicted = tokenizer.decode(predict_answer_tokens)
print(answer_predicted)

# attention
attentions = outputs.attentions
attention_weights = attentions[-1]
# 最後の層のattentionを取得したい
print(attention_weights.size())
# [batch_size, num_heads. sequence_length, sequence_length]
# print(f'attentions: {attention_weight}')
    
# from run_qa import prepare_validation_features

# predict_dataset = predict_examples.map(
#    prepare_validation_features,
#    batched=True,
#    # num_proc=data_args.preprocessing_num_workers,
#    remove_columns=column_names,
#    # load_from_cache_file=not data_args.overwrite_cache,
#    desc="Running tokenizer on prediction dataset",
#)


# prediction
logger = logging.getLogger(__name__)
logger.info("*** Predict ***")

cuda:0


Using custom data configuration default-13ffdd2859327b95
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-13ffdd2859327b95/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

志望 動機 を 答える 際 の ポイント は ？ 
志望 動機 を 答える 際 の ポイント は 、 なぜ その 企業 で なけれ ば なら ない か を 徹底的 に 突き詰める こと 。 この 企業 で なけれ ば なら ない という 熱意 が 強く 伝われ ば 伝わる ほど 、 面接 官 の 印象 は アップ し ます 。 志望 動機 の 考え方 と 実際 の 作り方 は 、 下記 記事 を 参考 に し て ください 。 
torch.Size([1, 125])
なせ その 企 業 て なけれ は なら ない か を 徹 底 的 に 突 き 詰 める こと
torch.Size([1, 12, 125, 125])


In [25]:
# the part of attention visualization
def highlight(word, attn):
    html_color = '#%02X%02X%02X' % (255, int(255*(1 - attn)), int(255*(1 - attn)))
    return '<span style="background-color: {}">{}</span>'.format(html_color, word)

def mk_html(index, inputs, preds, attention_weight):  
    sentence = inputs[index]
    #pred
    
    # label_str = "label_str"
    pred_str = preds
    
    html = "予測回答: {}<br>".format(pred_str)
    
    # 文章の長さ分のzero tensorを宣言
    seq_len = inputs.size()[1] # sequence_length
    device_cpu = torch.device("cpu")
    all_attens = torch.zeros(seq_len).to(device_cpu)
    
    for i in range(12):
        all_attens += attention_weight[index, i, 0, :]
        
    for word, attn in zip(sentence, all_attens):
        #if tokenizer.convert_ids_to_tokens([word.tolist()])[0] == "[SEP]":
        #    break
        html += highlight(tokenizer.convert_ids_to_tokens([word.numpy().tolist()])[0], attn)
    html += "<br><br>"
    return html

from IPython.display import display, HTML
html_output = mk_html(
    0,
    inputs.input_ids,
    answer_predicted,
    attentions[-1]
)
display(HTML(html_output))